# Sistemas de Recomendación

### Inteligencia Artificial

**Analítica de Negocio**

**UCEMA**

### Modelos de Filtrado Colaborativo basados en latentes o descomposiciones factoriales

## 6.1. Introdución

En los modelos de factores latentes los usuarios y los items son mapeados a un espacio de menor dimensión. Su nueva representación vectorial es este espacio se denomina latente. Veamos cómo sería este proceso:



Ésta representación latente tiene varias ventajas:
1. Al tener la representación latente de los usuarios e items, podemos reconstruir (o estimar) la matriz de puntuaciones


2. Esta nueva representación de los usuarios e items pueden ayudar a interpretar las predicciones

Y ¿cómo podemos obtener esta representación latente? Con el algoritmo **Alternating Least Squares** (ALS)

El algoritmo ALS trata de aproximar la matriz puntuaciones factorizándola como el producto de dos matrices:

$$ R = X * Y $$

donde X e Y son las matrices de factores latentes que describen las propiedades de cada usuario y cada item, respectivamente.

Estas matrices se calculan de tal manera que se minimiza el error cuadrático medio entre las prediciones y las puntuaciones reales sobre las puntuaciones conocidas (las de nuestro conjunto de entrenamiento):

$$ \min_{X, Y} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda \left( ||X||^2 + ||Y||^2  \right)
$$

donde P es el conjunto de elementos con puntuación conocida.

Para encontrar la solución de este problema, el algoritmo ALS comienza inicializando al azar las matrices de factores latentes $X$ e $Y$ y, luego, considerando conocidos los valores de $Y$, optimiza los valores de $X$ de tal manera que se minimiza:

$$ \min_{X} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda ||X||^2 $$

Luego, con los factores latentes obtenidos para los usuarios ($X$), optimiza el valor de las latentes de los items:
$$ \min_{Y} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda ||Y||^2 $$

E itera sobre estos dos pasos hasta que el algoritmo converge.

El hecho de ir alternado entre un problema en $X$ y un problema en $Y$ es lo que hace que este método se llame *Alternating Least Squares*.

Este método es un sistema de filtrado colaborativo con modelo, ya que realiza un entrenamiento previo donde calcula las latentes y para hacer prediciones solo necesita hacer el producto de las latentes asociadas (no necesita volver a acceder a toda la matriz de puntuaciones).

### Recomendación por mínimos cuadrados alternos (ALS)

Utilizaremos Apache Spark que es un motor de procesamiento distribuido que permite procesar grandes volúmenes de datos de manera eficiente y escalable. Proporciona abstracciones de datos como RDD, DataFrames y Datasets, y ofrece un conjunto de operaciones para transformar y analizar datos.


Spark se utiliza ampliamente en una variedad de aplicaciones de procesamiento de datos a gran escala, como análisis de datos, aprendizaje automático (machine learning), procesamiento de lenguaje natural (NLP) y procesamiento de secuencias temporales. Su flexibilidad, escalabilidad y rendimiento lo convierten en una herramienta popular en el campo de la inteligencia artificial y el análisis de big data.



####Referencias

https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

### Importa librerías

In [1]:
#https://grouplens.org/datasets/movielens/

In [2]:
!pip install pyspark

In [3]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

MovieLens es un sistema de recomendación y un sitio web de comunidad virtual que recomienda películas a sus usuarios en función de sus preferencias cinematográficas mediante filtrado colaborativo.

El conjunto de datos MovieLens 100M se extrae del sitio web MovieLens, que personaliza la recomendación al usuario basándose en las valoraciones que éste hace de las películas. Para comprender mejor el concepto de sistema de recomendación, trabajaremos con este conjunto de datos.
.

Hay 2 tuplas, películas y valoraciones, que contienen variables como MovieID::Genre::Title y UserID::MovieID::Rating::Timestamp respectivamente.

Vamos a cargar los datos y explorar los datos. Para cargar los datos como un marco de datos de spark, importa pyspark e instala una sesión de spark

### Inicializar la sesión de spark

In [4]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

# 1. Cargo data

In [5]:
from pyspark.sql.functions import split

# Leer como texto plano
raw_movies = spark.read.text("/content/sample_data/movies.dat")

# Separar las columnas por "::"
movies = raw_movies.withColumn("split", split(raw_movies["value"], "::")) \
                   .selectExpr("split[0] as movieId", "split[1] as title", "split[2] as genres")

movies.show(5)



+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [6]:
from pyspark.sql.functions import split

# Leer como texto
raw_ratings = spark.read.text("/content/sample_data/ratings.dat")

# Separar las columnas por "::"
ratings = raw_ratings.withColumn("split", split(raw_ratings["value"], "::")) \
                     .selectExpr("split[0] as userId",
                                 "split[1] as movieId",
                                 "split[2] as rating",
                                 "split[3] as timestamp")

ratings.show(5)


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    122|     5|838985046|
|     1|    185|     5|838983525|
|     1|    231|     5|838983392|
|     1|    292|     5|838983421|
|     1|    316|     5|838983392|
+------+-------+------+---------+
only showing top 5 rows



In [7]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [8]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    122|   5.0|
|     1|    185|   5.0|
|     1|    231|   5.0|
|     1|    292|   5.0|
|     1|    316|   5.0|
|     1|    329|   5.0|
|     1|    355|   5.0|
|     1|    356|   5.0|
|     1|    362|   5.0|
|     1|    364|   5.0|
|     1|    370|   5.0|
|     1|    377|   5.0|
|     1|    420|   5.0|
|     1|    466|   5.0|
|     1|    480|   5.0|
|     1|    520|   5.0|
|     1|    539|   5.0|
|     1|    586|   5.0|
|     1|    588|   5.0|
|     1|    589|   5.0|
+------+-------+------+
only showing top 20 rows



In [9]:
# Join both the data frames to add movie data into ratings
movie_ratings = ratings.join(movies, ['movieId'], 'left')
movie_ratings.show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|    122|     1|   5.0|    Boomerang (1992)|      Comedy|Romance|
|    185|     1|   5.0|     Net, The (1995)|Action|Crime|Thri...|
|    231|     1|   5.0|Dumb & Dumber (1994)|              Comedy|
|    292|     1|   5.0|     Outbreak (1995)|Action|Drama|Sci-...|
|    316|     1|   5.0|     Stargate (1994)|Action|Adventure|...|
|    329|     1|   5.0|Star Trek: Genera...|Action|Adventure|...|
|    355|     1|   5.0|Flintstones, The ...|Children|Comedy|F...|
|    356|     1|   5.0| Forrest Gump (1994)|Comedy|Drama|Roma...|
|    362|     1|   5.0|Jungle Book, The ...|Adventure|Childre...|
|    364|     1|   5.0|Lion King, The (1...|Adventure|Animati...|
|    370|     1|   5.0|Naked Gun 33 1/3:...|       Action|Comedy|
|    377|     1|   5.0|        Speed (1994)|Action|Romance|Th...|
|    420| 

## Calculo  "parsity"

En los problemas del mundo real, se espera que la matriz de utilidad sea muy dispersa, ya que cada usuario sólo puntua una pequeña fracción de artículos entre el vasto conjunto de opciones disponibles. El problema del arranque en frío puede surgir cuando se añade un nuevo usuario o un nuevo artículo, ya que ambos carecen de historial de valoraciones. La dispersión puede calcularse de la siguietne manera:

In [10]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.62% empty.


## Interpretando ratings

In [11]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
| 14463| 5169|
|  3817| 4165|
| 19635| 4165|
|  6757| 3414|
| 19379| 3202|
|  7795| 3187|
|  8811| 3164|
| 14134| 2753|
| 10858| 2533|
|  1860| 2529|
| 17438| 2422|
| 16681| 2353|
|  5134| 2337|
| 10759| 2227|
| 11760| 2213|
| 23858| 2206|
| 16227| 2176|
| 22842| 2170|
|  5678| 2152|
| 16003| 2145|
+------+-----+
only showing top 20 rows



In [12]:
# Group data by u MovieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    593|11372|
|    296|11366|
|    356|11169|
|    480|10536|
|    318|10179|
|    260| 9564|
|    110| 9525|
|    589| 9440|
|    457| 9232|
|      1| 8968|
|    780| 8874|
|    150| 8778|
|    592| 8584|
|    527| 8513|
|   1210| 8459|
|    590| 8441|
|     32| 8190|
|    380| 8138|
|    608| 8105|
|     50| 7926|
+-------+-----+
only showing top 20 rows



## 1. Construir un modelo ALS

Para construir el modelo especifique explícitamente las columnas.

Establezca no negativo como "Verdadero", ya que estamos buscando valoraciones superiores a 0.
El modelo también ofrece una opción para seleccionar valoraciones implícitas pero como estamos trabajando con valoraciones explícitas, configúrela como "Falso".

Cuando se utilizan divisiones aleatorias simples como en Spark's CrossValidator o TrainValidationSplit, es muy común encontrar usuarios y/o elementos en el conjunto de evaluación que no están en el conjunto de entrenamiento.

Por defecto, Spark asigna predicciones NaN durante ALSModel.transform cuando un factor de usuario y/o elemento no está presente en el modelo.

Esto no es deseable durante la validación cruzada, ya que cualquier valor NaN predicho dará lugar a resultados NaN para la métrica de evaluación (por ejemplo, cuando se utiliza RegressionEvaluator). Esto hace imposible la selección del modelo.

Spark permite a los usuarios establecer el parámetro coldStartStrategy en "drop" para eliminar cualquier fila del DataFrame de predicciones que contenga valores NaN. La métrica de evaluación se calculará entonces sobre los datos no NaN y será válida.

In [21]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, explode, count

# 1. Filtrar usuarios e ítems con suficiente actividad
user_freq = ratings.groupBy("userId").agg(count("movieId").alias("count"))
filtered_users = user_freq.filter("count >= 50").select("userId")

item_freq = ratings.groupBy("movieId").agg(count("userId").alias("count"))
filtered_items = item_freq.filter("count >= 50").select("movieId")

ratings_filtered = ratings.join(filtered_users, on="userId")\
                          .join(filtered_items, on="movieId")



In [22]:

# 2. Entrenar modelo ALS con validación cruzada optimizada
#rank es el número de factores latentes en el modelo (por defecto 10).

#regParam especifica el parámetro de regularización en ALS (por defecto 1.0).

#maxIter es el número máximo de iteraciones a ejecutar (por defecto 10).

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 50]) \
    .addGrid(als.regParam, [0.05, 0.1]) \
    .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Split train/test
train, test = ratings_filtered.randomSplit([0.8, 0.2], seed=42)

# Entrenar el modelo
cv_model = cv.fit(train)
best_model = cv_model.bestModel


In [23]:
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  4


In [24]:

# 3. Evaluación del mejor modelo
print("**Best Model**")
print("  Rank:", best_model._java_obj.parent().getRank())
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

test_predictions = best_model.transform(test)
rmse_score = evaluator.evaluate(test_predictions)
print(f"RMSE en test: {rmse_score:.4f}")

**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.05
RMSE en test: 0.7867


In [25]:
test_predictions.show()

+-------+------+------+----------+
|movieId|userId|rating|prediction|
+-------+------+------+----------+
|      1|    18|   3.0|  3.469595|
|      1|    36|   4.0|  4.081869|
|      1|    47|   3.0| 3.9295778|
|      1|    76|   4.0| 3.9868145|
|      1|   100|   1.5| 2.6405373|
|      1|   114|   4.0| 3.5853238|
|      1|   136|   4.0|  3.629431|
|      1|   149|   3.5| 2.8714585|
|      1|   178|   4.0|  3.765496|
|      1|   179|   5.0| 4.6160173|
|      1|   182|   4.0| 4.0201745|
|      1|   194|   5.0| 3.6481233|
|      1|   198|   4.5|   4.19569|
|      1|   209|   4.0| 4.0759835|
|      1|   221|   3.5| 4.1343985|
|      1|   274|   5.0| 4.5925393|
|      1|   296|   3.5| 3.5132084|
|      1|   389|   3.0| 3.3172297|
|      1|   428|   5.0|  3.894589|
|      1|   438|   5.0|  3.908434|
+-------+------+------+----------+
only showing top 20 rows



El RMSE del mejor modelo es de 0,78, lo que significa que, de media, el modelo predice 0,78 valores por encima o por debajo de la matriz de valoraciones original. Ten en cuenta que la factorización matricial desentraña patrones que los humanos no podemos, por lo que puedes encontrarte con que las valoraciones de unos pocos usuarios están un poco desviadas en comparación con otras.

## Make Recommendations

Vamos a seguir adelante y hacer recomendaciones basadas en nuestro mejor modelo.

La función recommendForAllUsers(n) en ALS toma n recomendaciones. Vamos con 10 recomendaciones para todos los usuarios.

In [26]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     5|[{824, 5.007306},...|
|    12|[{260, 4.4662976}...|
|    13|[{55872, 4.545727...|
|    17|[{7089, 5.335032}...|
|    19|[{1197, 4.7731147...|
|    34|[{110, 3.9201097}...|
|    35|[{296, 4.6435943}...|
|    37|[{296, 4.6654134}...|
|    40|[{296, 5.0129704}...|
|    41|[{1035, 4.5233397...|
+------+--------------------+



Las recomendaciones se generan en un formato fácil de usar en pyspark. Como se ve en la salida de arriba, las recomendaciones se guardan en un formato de matriz con el id de la película y las calificaciones.

Para hacer estas recomendaciones fáciles de leer y comparar para comprobar si las recomendaciones tienen sentido, vamos a querer añadir más información como el nombre de la película y el género, a continuación, explotar matriz para obtener filas con recomendaciones individuales.

In [27]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     5|    824| 5.007306|
|     5|   6987| 4.900994|
|     5|   5105|4.8829846|
|     5|   8195| 4.822296|
|     5|  26150| 4.804935|
|     5|   1354| 4.797269|
|     5|   1348|4.7527957|
|     5|   2360|4.7525716|
|     5|   2132| 4.733651|
|     5|   3739|4.7319193|
+------+-------+---------+



## ¿Tienen sentido las recomendaciones?
Combinemos los nombres de las películas y los géneros en la matriz de recomendaciones para facilitar la interpretación.

Para comprobar si las recomendaciones tienen sentido, se une el nombre de la película y el género a la tabla anterior.

Escojamos aleatoriamente al usuario número 100 para comprobar si las recomendaciones tienen sentido.

Recomendaciones de ALS para el usuario 100th:




In [28]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|    296|   100| 3.690885| Pulp Fiction (1994)|  Comedy|Crime|Drama|
|    858|   100|3.6627133|Godfather, The (1...|         Crime|Drama|
|   1221|   100|3.6069632|Godfather: Part I...|         Crime|Drama|
|   1089|   100|3.6009707|Reservoir Dogs (1...|Crime|Drama|Thriller|
|   1213|   100| 3.586363|   Goodfellas (1990)|         Crime|Drama|
|     50|   100|3.4735515|Usual Suspects, T...|Crime|Mystery|Thr...|
|   2360|   100|3.4523048|Celebration, The ...|               Drama|
|   1208|   100|3.4408054|Apocalypse Now (1...|    Action|Drama|War|
|   1228|   100|3.4398232|  Raging Bull (1980)|               Drama|
|   5515|   100| 3.434678|Songs From the Se...|               Drama|
+-------+------+---------+--------------------+--------------------+



Preferencia real del usuario 100th:

In [29]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|    318|   100|   4.5|Shawshank Redempt...|               Drama|
|   3996|   100|   4.5|Crouching Tiger, ...|Action|Adventure|...|
|   1233|   100|   4.5|Boat, The (Das Bo...|    Action|Drama|War|
|   6807|   100|   4.5|Monty Python's Th...|              Comedy|
|    953|   100|   4.0|It's a Wonderful ...|Drama|Fantasy|Rom...|
|   1073|   100|   4.0|Willy Wonka & the...|Children|Comedy|F...|
|    904|   100|   4.0|  Rear Window (1954)|    Mystery|Thriller|
|   1089|   100|   4.0|Reservoir Dogs (1...|Crime|Drama|Thriller|
|    296|   100|   4.0| Pulp Fiction (1994)|  Comedy|Crime|Drama|
|   2176|   100|   4.0|         Rope (1948)|Crime|Drama|Thriller|
+-------+------+------+--------------------+--------------------+



La película recomendada al usuario número 100 pertenece principalmente a los géneros de drama y crimen, y las películas preferidas por el usuario, como se ve en la tabla anterior, coinciden mucho con estos géneros.